In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, Embedding, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Input
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# 1. 数据生成：模拟分类任务的序列数据
n_samples = 1000
sequence_length = 20
vocab_size = 50  # 词汇表大小
n_classes = 2

In [3]:
# 随机生成序列数据和分类标签
X = np.random.randint(0, vocab_size, size=(n_samples, sequence_length))  # 输入序列
y = np.random.randint(0, n_classes, size=(n_samples,))  # 标签

In [4]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# 2. 定义 Transformer 模型
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),  # 前馈网络
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        # 多头注意力
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # 残差连接 + Layer Norm
        
        # 前馈网络
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # 残差连接 + Layer Norm

In [12]:
# 3. 构建完整的 Transformer 模型
def build_transformer_model(sequence_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes):
    inputs = Input(shape=(sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)
    x = embedding_layer(inputs)
    
    # 添加 Transformer Block
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    
    # 全局池化 + 分类头
    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    
    return Model(inputs=inputs, outputs=outputs)

In [13]:
# 模型参数
embed_dim = 32  # 嵌入维度
num_heads = 2   # 注意力头数量
ff_dim = 64     # 前馈网络隐藏层维度
num_classes = n_classes

In [14]:
model = build_transformer_model(sequence_length, vocab_size, embed_dim, num_heads, ff_dim, num_classes)

In [15]:
# 4. 编译模型
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [16]:
# 5. 训练模型
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2, verbose=1)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5324 - loss: 0.7059 - val_accuracy: 0.5063 - val_loss: 0.7052
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5135 - loss: 0.6977 - val_accuracy: 0.5375 - val_loss: 0.6928
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5607 - loss: 0.6875 - val_accuracy: 0.5250 - val_loss: 0.6902
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6072 - loss: 0.6772 - val_accuracy: 0.5125 - val_loss: 0.6997
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5762 - loss: 0.6741 - val_accuracy: 0.5125 - val_loss: 0.6957
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5670 - loss: 0.6771 - val_accuracy: 0.5063 - val_loss: 0.6966
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5701 - loss: 0.6726 - val_accuracy: 0.5000 - val_loss: 0.7227
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6048 - loss: 0.6674 - val_accuracy: 0.5063 - val_loss

In [17]:
# 6. 测试模型
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Test Accuracy: 0.51
